In grammars, there are two major sets for each non-terminal, named first & follow sets. These sets are required for the compiler to be able to recognize the rules of a language. In this notebook we are going to accomplish the first and follow sets of a given grammar.
First, we need to read the grammar from a file:
Note that for this project, the given grammar must be written in this example's format:
<center>
    S => AbcaD | BcdC | ab
    A => Aa | ε | abB
    B => bAC | ε
    C => c | ε
    D => a | ε
</center>

In [12]:
f = open('grammar.txt', 'r')
lines = f.readlines()
f.close()

Now we define our non-terminal class, which contains:
1. symbol
2. rules: set of rules a non-terminal can be replaced with.
3. first: first set of the non-terminal
4. follow: follow set of the non-terminal
5. firstdepen: indicates what other non-terminals does a non-terminal's first set depend on. (for example S => ABabC would tell us the first set of S depends on the first set of A)
6. followdepen: indicates what other non-terminals does a non-terminal's follow set depend on.
Here's the class definition:

In [13]:
class NonTerminal:
    def __init__(self, symbol):
        self.symbol = symbol
        self.rules = []
        self.first = []
        self.firstdepen = []
        self.followdepen = []
        self.follow = []

As mentioned earlier, each non-terminal contains a set of rules to be replaced with when expanding the language. we have defined these rules to be of type rule. Here is the definition of class Rule:

In [14]:
class Rule:
    def __init__(self, s: str, nont: NonTerminal):
        self.s = s
        self.nont = nont
        self.nont.rules.append(self)

A rule contains a string s, which holds all the terminals/non-terminals of the rule in n string. It also has nont of type NonTerminal, which is the rule's co-related non-terminal.
self.nont.rules.append(self) makes sure the non-terminal and the rule are connected right when the rule is instantiated.
Now we need to instantiate out non-terminals and rules:

In [15]:
nonterminals = []


for line in lines:
    line = line.replace('\n', '')
    rules = []
    symbol, rule = line.split(' => ')
    nonterminal = NonTerminal(symbol)
    nonterminals.append(nonterminal)
    rules.extend(rule.split(' | '))
    for rule in rules:
        r = Rule(rule, nonterminal)
    nonterminals[0].follow.append('$')

Since S is our starting non-terminal, we add $ to its follow set.
Now we have to obtain the first set of our non-terminals.
There are a few rules in creating the first set of a non-terminal:
1. first of a terminal (lowercase letter) is the terminal itself.
2. if there's a rule in non-terminal1 that starts with a non-terminal2, all the items in the first set of non-terminal2 are added to the first set of non-terminal1, except for the epsilon. for example in S => ABcD, the first set of S will be extended by adding the items in the first set of A.
3. in the previous example, if there's a rule in A where A => ε, then we add the items of the first set of B to S too, since A could be replaced with ε and S could be started with B, and so on.
Here's the implementation of the method first which accepts a non-terminal nt as its argument:


In [16]:
def first(nt):
    for rule in nt.rules:
        if rule.s[0].islower():
            nt.first.append(rule.s[0])
        elif rule.s[0].isupper():
            for c in rule.s:
                if c.isupper():
                    nt2 = next((n for n in nonterminals if n.symbol == c), None)
                    if nt2 not in nt.firstdepen:
                        nt.firstdepen.append(nt2)
                        nt.firstdepen.extend(nt2.firstdepen)
                        d = first(nt2)
                        nt.first.extend([x for x in d if x != 'ε'])
                        if 'ε' in d:
                            if c == rule.s[-1]:
                                nt.first.append('ε')
                            continue
                        else:
                            break
                    else:
                        if 'ε' in nt2.first:
                            continue
                        else:
                            break
                else:
                    nt.first.append(c)
                    break
        else:
            nt.first.append('ε')
    nt.first = list(set(nt.first))
    return nt.first

In this method create almost all non-terminals first sets, except for the ones that are not accessed by the other non-terminals. To make sure that all our terminals have their first sets up and ready, we added this block of code to obtain the first sets of the non-terminals that were not initialized earlier.

In [17]:
for nt in nonterminals:
    if len(nt.first) == 0:
        first(nt)

Let's test out method on the example grammar at the beginning of the notebook.

In [18]:
for nt in nonterminals:
    print(f'First({nt.symbol}): ', '{', end='')
    for i in range(len(nt.first)):
        print(f' {nt.first[i]}', end='')
        if i != len(nt.first) - 1:
            print(',', end='')
        else:
            print(' }')

First(S):  { c, a, b }
First(A):  { a, ε }
First(B):  { ε, b }
First(C):  { c, ε }
First(D):  { a, ε }


Now we need to obtain the follow sets of non-terminals. Here's the implementation of the method follow():

In [19]:
def follow():
    for nt in nonterminals:
        for rule in nt.rules:
            for j in range(len(rule.s)):
                if rule.s[j].isupper():
                    nt2 = next(n for n in nonterminals if n.symbol == rule.s[j])
                    if j == len(rule.s) - 1:
                        nt2.followdepen.append(nt)
                        if 'ε' in nt2.first:
                            for k in range(j - 1, 0):
                                if rule.s[k].isupper():
                                    nt4 = next(n for n in nonterminals if n.symbol == rule.s[k])
                                    nt4.followdepen.append(nt)
                                else: break
                        continue
                    else:
                        for c in range(j + 1, len(rule.s)):
                            if rule.s[c].isupper():
                                nt3 = next(n for n in nonterminals if n.symbol == rule.s[c])
                                nt2.follow.extend([bigh for bigh in nt3.first if bigh != 'ε'])
                                if 'ε' in nt3.first:
                                    continue
                                else:
                                    break
                            else:
                                nt2.follow.append(rule.s[c])
                                break
    
    for nt in nonterminals:
        nt.follow = list(set(nt.follow))
        if nt.symbol in nt.follow:
            nt.follow.remove(nt.symbol)

Now we know each non-terminal depends on which other ones for its follow set. let's update the follow sets:

In [20]:
def upd_follow(nt: NonTerminal):
    for depen in nt.followdepen:
        nt.follow.extend(depen.follow)
    revdep = [n for n in nonterminals if nt in n.followdepen]
    for dep in revdep:
        dep.follow.extend(nt.follow)
        dep.follow = list(set(dep.follow))
    nt.follow = list(set(nt.follow))

In [21]:
follow()
for nt in nonterminals:
    upd_follow(nt)

In [22]:
for nt in nonterminals:
    print(f'Follow({nt.symbol}): ', '{', end='')
    for i in range(len(nt.follow)):
        print(f' {nt.follow[i]}', end='')
        if i != len(nt.follow) - 1:
            print(',', end='')
        else:
            print(' }')

Follow(S):  { $ }
Follow(A):  { c, a, b }
Follow(B):  { c, a, b }
Follow(C):  { c, a, $, b }
Follow(D):  { $ }
